In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'llm-datasets:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4437625%2F7618922%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240227%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240227T114347Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D13b09b03bf50ad8d74e05c6ac6da319be3ad0340e28c0a75996d522a0c06a796a172b1092c94d1539ff8f88dc1f70738064ace892cd37f01081066d513332661ae31cc850e65cd0a857b6d74e198ac9211f30cba4f00f1c2a6b6080bc4cea888c11abd3154bd5d1bca0fdfd200450bb15d50a2a43dcccb186c9f8c94154a48a4e1a11775d337b8450b919a22acaaa7d857dada3ae0537557f82df934c407d2670c1799181d2b81dea82066e9e41a29575ea7f65ad2a8de51876e9d41da396d33e7bdecfbbf95a8003ed92626a362e1d851525cb521c9e51d9fb6f4d6bb4f93bab7a52e69c683e19cdf4a5c65f9a833f83a68d358dd6184e7815d1f1bb183a6fa,llm-model:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4503294%2F7711889%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240227%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240227T114347Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3e80627b72ba2271578dbbf149d5182fc1cbb71a26145491de6c12833121d5b069d526decb9db08faef84b19c4e31770c73455a5e6c1dc5cf42af01197d57839a14e7960f4e34a4568cfb2e58d0805b9fb79e6831d0d152c815c9cf684b1126a8ad9fae4bb11dceda1529fa1fea6431e9b7651de99b1ff3a2a3c278f0a37e813104429852e27432bd9d5a7cbabccd81ee66abe222ea50f6faf8a0a4e2ebf956b4e0f2bd1a9f8a6fe7eff700541739731a8a95cad1c752f502b46744d1e36398a2ca2b223b81f6afdbaad5b8d2b6024949774cc8e8275b9d8418da176904ea45d875ff2d7f8d50597f846d925955c5c5ef1af68d371b2f481dd8e398e146006df'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-datasets/sample_submission.csv
/kaggle/input/llm-datasets/train.csv
/kaggle/input/llm-datasets/test.csv


In [ ]:
!pip install gdown
!pip install kaggle --upgrade
!pip install wandb
!pip install peft
!pip install bitsandbytes
# !pip install transformers accelerate bitsandbytes
#!pip install git+https://github.com/zhanghang1989/PyTorch-Encoding/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.6.6-py3-none-any.whl size=111943 sha256=87cb51c1f06b8add5ce3955093a82c7b4948c4b80416e4cc87956e61d408e8bd
  Stored in directory: /root/.cache/pip/wheels/53/34/8c/8ca3450d17206d9e37e1ee3aeb47cbb2873d22a9e0c60eb137
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.6.3
    Uninstalling kaggle-1.6.3:
      Successfully uninstalled kaggle-1.6.3


In [ ]:
import gdown
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import lightgbm as lgb
import bisect
import torch
import torch.nn as nn
import re
import nltk
import torch
import transformers
import bitsandbytes as bnb
from transformers import PreTrainedTokenizerFast, AdamW, AutoModelForCausalLM, BitsAndBytesConfig, AutoModel
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from tqdm import tqdm
from transformers import BertTokenizerFast, GPT2LMHeadModel
from transformers import ElectraModel, ElectraTokenizer
#from parallel import DataParallelModel, DataParallelCriterion
from transformers import AutoTokenizer, AutoModelForCausalLM
from nltk.corpus import stopwords
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
!git clone https://github.com/ttony0321/stopwords.txt.git
#os.environ['WANDB_DISABLED'] = "false"

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


Cloning into 'stopwords.txt'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), 4.29 KiB | 4.29 MiB/s, done.


In [ ]:
#https://drive.google.com/file/d/1ySDZrZlDfkxdGOt_h4zofIHJNhkUq5oB/view?usp=sharing
#!gdown 1ySDZrZlDfkxdGOt_h4zofIHJNhkUq5oB
#if gdown doesn't work
#!unzip /kaggle/working/open.zip

In [ ]:
!nvidia-smi

Tue Feb 27 08:52:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              25W / 250W |      2MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1"  # Set the GPU 2 to use

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [ ]:
# train = pd.read_csv('/kaggle/working/train.csv')
# test = pd.read_csv('/kaggle/working/train.csv')
#if not gdown
train = pd.read_csv('/kaggle/input/llm-datasets/train.csv')
test = pd.read_csv('/kaggle/input/llm-datasets/test.csv')

In [ ]:
train.head()

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
0,TRAIN_000,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,TRAIN_001,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,건축구조,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."
2,TRAIN_002,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,건축구조,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...
3,TRAIN_003,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,건축구조,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ..."
4,TRAIN_004,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,건축구조,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...


In [ ]:
train['category'].value_counts()

category
마감재       272
인테리어      123
시공        111
마감하자       60
건축구조       31
기타         27
타 마감하자     20
Name: count, dtype: int64

In [ ]:
stopwords_kr = '/kaggle/working/stopwords.txt/stopwords.txt'
with open(stopwords_kr, 'r') as file:
    read_line = file.read()


In [ ]:
stop_words = read_line.replace('\n', ' ')
stop_words = set(stop_words.split(' '))

In [ ]:
#preprocess
def remove_spec(text):
    #text = re.sub('[^가-힣a-z]', ' ', text)
    text = re.sub(r'[^가-힣A-Za-z0-9 ]', '', text)#특수문자 제거
    text = re.sub(r'\s+', ' ', text).strip()#공백 하나
    return text
#stop words
def stopw(text):
    tokens = text.split()
    tokens = [word for word in tokens if not word in stop_words]
    cleaned_text = " ".join(tokens)
    return cleaned_text


In [ ]:
train['질문_1'] = train['질문_1'].apply(remove_spec)
train['질문_2'] = train['질문_2'].apply(remove_spec)

train['답변_1'] = train['답변_1'].apply(remove_spec)
train['답변_2'] = train['답변_2'].apply(remove_spec)
train['답변_3'] = train['답변_3'].apply(remove_spec)
train['답변_4'] = train['답변_4'].apply(remove_spec)
train['답변_5'] = train['답변_5'].apply(remove_spec)

train['질문_1'] = train['질문_1'].apply(stopw)
train['질문_2'] = train['질문_2'].apply(stopw)

train['답변_1'] = train['답변_1'].apply(stopw)
train['답변_2'] = train['답변_2'].apply(stopw)
train['답변_3'] = train['답변_3'].apply(stopw)
train['답변_4'] = train['답변_4'].apply(stopw)
train['답변_5'] = train['답변_5'].apply(stopw)

In [ ]:
train.head()

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
0,TRAIN_000,면진장치가 뭐야,면진장치에 사용되는 주요 기술은 무엇인가요,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고 진동을 ...,면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,TRAIN_001,내진설계의 종류 알려줘,내진설계에는 종류가 있는지 자세히 알려주실 있나요,건축구조,내진 설계의 종류로 내진구조 제진구조 면진구조가 있습니다,내진설계에는 내진구조 제진구조 면진구조가 있습니다 내진구조는 건물 구조물이 지진의 ...,내진설계에는 주로 내진구조 제진구조 면진구조의 세 가지 종류가 있습니다 이들은 지진...,내진설계에는 주로 내진구조 제진구조 면진구조가 사용됩니다 내진구조는 건물 구조물 전...,내진 설계에는 다양한 종류가 있지만 대표적으로 내진구조 제진구조 면진구조가 있습니다...
2,TRAIN_002,철골구조의 장점이 뭐야,철골구조의 장점을 알려줘,건축구조,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 고층 건물의 건축이 가능한 것이 장...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 고층 건물의 건축이 가능하다...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 고층 건물의 건축이 용이하다는 ...,철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고 고층 건물...,철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 있는 ...
3,TRAIN_003,철골철근 콘크리트 구조가 뭐야,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요,건축구조,철근철골콘크리트는 철골과 철근 콘크리트를 사용하는 건축 구조입니다 철골은 강하고 가...,철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근 콘크리트를 사용...,철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 사용되는 방식으로 철골을...,철골철근콘크리트 구조는 철골과 철근 콘크리트를 사용하여 만들어지는 건축 구조입니다 ...,철골철근 콘크리트 구조는 강철 골조와 강철 철근 콘크리트를 사용하여 건축물을 구성하...
4,TRAIN_004,철골구조는 방식이 있어,철골구조의 다양한 방식이 무엇인가요,건축구조,철골구조는 일반철골구조와 경량철골구조가 있습니다,철골구조는 일반철골구조와 경량철골구조가 있습니다 일반철골구조는 주로 대형 건물이나 ...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다 이들은 건축 시스템에 선적...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다 최근에는 고층 건물에서 사용...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다 일반철골구조는...


In [ ]:
train.isnull().sum()

id          0
질문_1        0
질문_2        0
category    0
답변_1        0
답변_2        0
답변_3        0
답변_4        0
답변_5        0
dtype: int64

In [ ]:
#skt_tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2', eos_token='</s>', pad_token='<pad>')
#kakao_tokenizer = AutoTokenizer.from_pretrained('kakaobrain/kogpt', eos_token='[EOS]')
polyglot_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-12.8B", eos_token='<|endoftext|>')
# Klm_tokenizer = transformers.AutoTokenizer.from_pretrained("quantumaikr/KoreanLM-1.5b", eos_token='</s>')
# dot_42_tokenizer = transformers.AutoTokenizer.from_pretrained("42dot/42dot_LLM-PLM-1.3B", eos_token="<|endoftext|>")

#polyglot_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-3.8b", eos_token='<|endoftext|>')
#electra_tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-generator", eos_token='<|endoftext|>')


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


tokenizer_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/529k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.26M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
max_length = 128
formatted_data = []
for _, row in tqdm(train.iterrows()):
    for q_col in ['질문_1', '질문_2']:
        for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            # 질문과 답변 쌍을 </s> token으로 연결
            input_text = row[q_col] + skt_tokenizer.eos_token + row[a_col]
            input_ids = skt_tokenizer.encode(input_text, return_tensors='pt', padding='max_length', truncation=True, max_length=max_length)
            #input_ids = skt_tokenizer.encode(input_text, return_tensors='pt')
            formatted_data.append(input_ids)
print('Done.')

644it [00:02, 277.30it/s]

Done.


In [ ]:
formatted_data = torch.cat(formatted_data, dim=0)

In [ ]:
formatted_data[0]

tensor([[ 9411,  8265, 37765, 46651,  7991,     1,  9411,  8265, 20725,  7374,
          9027,  7599,  9023, 14472, 15898, 14820, 33220, 36928, 10764,  9166,
         11818, 28037, 10090, 15898, 34062, 20725,  8153,  7172,  7182]])

In [ ]:
CFG = {
    'LR':5e-5,
    'EPOCHS':20,
    'BATCH_SIZE':8
}

In [ ]:
#Using Wandb
import wandb
import random
wandb.login()
wandb.init(
    project = 'LLM_deaplearning',
    config={
    "learning_rate": CFG['LR'],
    "architecture": "skt/kogpt2-base-v2",
    "dataset": formatted_data,
    "epochs": CFG['EPOCHS'],
    }
)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ttony0321. Use `wandb login --relogin` to force relogin


In [ ]:
model = AutoModelForCausalLM.from_pretrained('EleutherAI/polyglot-ko-12.8b')
print(model)

In [ ]:
#module 확인

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(model):

    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

lora_modules = find_all_linear_names(model)

In [ ]:
print(lora_modules)

In [ ]:
from torch.nn.parallel import DistributedDataParallel as DDP
# 4bit quantization
# load model
model_id = 'EleutherAI/polyglot-ko-12.8b'



#setting 4bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"":0},
)

#target modules support torch.nn.Linear`, `torch.nn.Embedding`, `torch.nn.Conv2d`, `transformers.pytorch_utils.Conv1D`.
model.gradient_checkpointing_enable()#GPU 사용량 줄이고 연산을 더하는 코드
model = prepare_model_for_kbit_training(model)
config = LoraConfig(#
    r=8,
    lora_alpha=32,
    #target_modules=["query_key_value"],
    target_modules=[
    "query_key_value",
    "dense",
    "dense_h_to_4h",
    "dense_4h_to_h"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)
model.to(device)
optimizer = AdamW(model.parameters(), lr = CFG['LR'])


In [ ]:
# 4bit quantization
import transformers
import wandb
trainer = transformers.Trainer(
    model=model,
    train_dataset=formatted_data,
    args=transformers.TrainingArguments(
        report_to='none',
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        #max_steps=50,
        learning_rate=CFG['LR'],
        fp16=True,
        logging_steps=10,
        output_dir="/kaggle/working/outputs",
        #optim="paged_adamw_8bit"
        optim='adamw_8bit'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(polyglot_tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

In [ ]:
%cd /kaggle/working/save
!ls

In [ ]:
from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "data.csv"):
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='model.safetensors')
from IPython.display import FileLink
#FileLink(r'model.safetensors')

In [ ]:
import zipfile
import os
import subprocess
from IPython.display import FileLink, display
def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))


In [ ]:
download_file('/kaggle/working/save/', 'model.safetensors')

In [ ]:
# 저장된 Fine-tuned 모델과 토크나이저 불러오기
model_dir = "/kaggle/working/save"
#model = GPT2LMHeadModel.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir, quantized=True)
model.to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_dir)

# Inference를 위한 test.csv 파일 로드
test = pd.read_csv('/kaggle/input/llm-datasets/test.csv')

# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
preds = []

# '질문' 컬럼의 각 질문에 대해 답변 생성
for test_question in tqdm(test['질문']):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    output_sequences = model.generate(
        input_ids=input_ids.to(device),
        max_length=300,
        temperature=0.9,
        top_k=1,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )

    # 생성된 텍스트(답변) 저장
    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        # 질문과 답변의 사이를 나타내는 eos_token (</s>)를 찾아, 이후부터 출력
        answer_start = full_text.find(tokenizer.eos_token) + len(tokenizer.eos_token)
        answer_only = full_text[answer_start:].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)

In [ ]:
!pip install sentence_transformers

In [ ]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape
submit = pd.read_csv('/kaggle/input/llm-datasets/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('/kaggle/working/baseline_submit.csv', index=False)